In [ ]:
import sympy as sp

# Defina as variáveis simbólicas
deltaL, Lo, deltaT, alpha = sp.symbols('deltaL Lo deltaT alpha')

# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_parcial_deltaL = sp.diff(alpha_expr, deltaL)

# Exiba o resultado
print(derivada_parcial_deltaL)


1/(Lo*deltaT)


**Testando aumotizar os cálculos direto do excel**

In [ ]:

import sympy as sp
import pandas as pd


dados = pd.read_excel('dados.xlsx')

# Defina as variáveis simbólicas
x, y, z = sp.symbols('x y z')

# Defina a expressão
expr = x**2 + y**2 + z**2


# Obtenha os valores do array para substituição
coluna1 = dados['x'].values
print(coluna1)
y_valor = dados['y'].values
print(y_valor)

# Substitua os valores diretamente na expressão
expr_substituida = expr.subs({x: coluna1[0], y: y_valor[0], z: 4})
print(expr_substituida)

[1 2 3]
[3 4 5]
26


Vou aplicar isso para as derivadas

In [2]:
import pandas as pd
import sympy as sp

# Carregue o arquivo Excel
dados2 = pd.read_excel('dados2.xlsx')

# Transforme os dados em um dicionário
dados_dict = dict(zip(dados2['variavel'], dados2['valor']))

# Exiba o dicionário
print(dados_dict)


# Defina as variáveis simbólicas
deltaL, Lo, deltaT, alpha = sp.symbols('deltaL Lo deltaT alpha')

# Defina a expressão para alpha
alpha_expr = deltaL / (Lo * deltaT)

# Calcule a primeira derivada parcial de alpha em relação a DeltaL
derivada_deltaL = sp.diff(alpha_expr, deltaL)

# Avalie a derivada no ponto específico
valor_da_derivada_em_ponto = derivada_deltaL.subs(dados_dict)

# Exiba o resultado
print("oi",valor_da_derivada_em_ponto)

{'deltaL': 0.23, 'Lo': 22.0, 'deltaT': 25.0, 'alpha': 0.00027}
oi 0.00181818181818182
